# Example-27: Normalize

In [1]:
# In this example normalized objective construction is illustrated

In [2]:
# Import

import torch
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
torch.set_printoptions(linewidth=128)

import matplotlib
from matplotlib import pyplot as plt
matplotlib.rcParams['text.usetex'] = True

from twiss import twiss

from ndmap.signature import chop
from ndmap.evaluate import evaluate
from ndmap.pfp import parametric_fixed_point

from model.library.drift import Drift
from model.library.quadrupole import Quadrupole
from model.library.sextupole import Sextupole
from model.library.dipole import Dipole
from model.library.line import Line

from model.command.wrapper import group
from model.command.wrapper import forward
from model.command.wrapper import inverse
from model.command.wrapper import normalize
from model.command.wrapper import Wrapper

In [3]:
# Define simple FODO based lattice using nested lines

DR = Drift('DR', 0.25)
BM = Dipole('BM', 3.50, torch.pi/4.0)

QF_A = Quadrupole('QF_A', 0.5, +0.20)
QD_A = Quadrupole('QD_A', 0.5, -0.19)
QF_B = Quadrupole('QF_B', 0.5, +0.20)
QD_B = Quadrupole('QD_B', 0.5, -0.19)
QF_C = Quadrupole('QF_C', 0.5, +0.20)
QD_C = Quadrupole('QD_C', 0.5, -0.19)
QF_D = Quadrupole('QF_D', 0.5, +0.20)
QD_D = Quadrupole('QD_D', 0.5, -0.19)

SF_A = Sextupole('SF_A', 0.25, 0.00)
SD_A = Sextupole('SD_A', 0.25, 0.00)
SF_B = Sextupole('SF_B', 0.25, 0.00)
SD_B = Sextupole('SD_B', 0.25, 0.00)
SF_C = Sextupole('SF_C', 0.25, 0.00)
SD_C = Sextupole('SD_C', 0.25, 0.00)
SF_D = Sextupole('SF_D', 0.25, 0.00)
SD_D = Sextupole('SD_D', 0.25, 0.00)

FODO_A = Line('FODO_A', [QF_A, DR, SF_A, DR, BM, DR, SD_A, DR, QD_A, QD_A, DR, SD_A, DR, BM, DR, SF_A, DR, QF_A], propagate=True, dp=0.0, exact=False, output=False, matrix=False)
FODO_B = Line('FODO_B', [QF_B, DR, SF_B, DR, BM, DR, SD_B, DR, QD_B, QD_B, DR, SD_B, DR, BM, DR, SF_B, DR, QF_B], propagate=True, dp=0.0, exact=False, output=False, matrix=False)
FODO_C = Line('FODO_C', [QF_C, DR, SF_C, DR, BM, DR, SD_C, DR, QD_C, QD_C, DR, SD_C, DR, BM, DR, SF_C, DR, QF_C], propagate=True, dp=0.0, exact=False, output=False, matrix=False)
FODO_D = Line('FODO_D', [QF_D, DR, SF_D, DR, BM, DR, SD_D, DR, QD_D, QD_D, DR, SD_D, DR, BM, DR, SF_D, DR, QF_D], propagate=True, dp=0.0, exact=False, output=False, matrix=False)

RING = Line('RING', [FODO_A, FODO_B, FODO_C, FODO_D], propagate=True, dp=0.0, exact=False, output=False, matrix=False)

In [4]:
# Set parametric mapping

ring, *_ = group(RING, 'FODO_A', 'FODO_D', ('ms', ['Sextupole'], None, None), ('dp', None, None, None), root=True)

In [5]:
# Construct normalized function

fn = normalize(ring, [(None, None), (-10.0, 10.0), (-0.01, 0.01)])

# Compare with original

fp = torch.tensor([0.001, 0.0005, -0.010, 0.0025], dtype=torch.float64)
ms = torch.tensor([1.0, -1.0, 0.5, 2.0, 4.0, -5.0, -1.0, 3.0], dtype=torch.float64)
dp = torch.tensor([0.005], dtype=torch.float64)

print(ring(fp, ms, dp))
print(fn(*forward([fp, ms, dp],  [(None, None), (-10.0, 10.0), (-0.01, 0.01)])))

tensor([ 0.0157, -0.0006, -0.0189, -0.0032], dtype=torch.float64)
tensor([ 0.0157, -0.0006, -0.0189, -0.0032], dtype=torch.float64)


In [6]:
# Set deviation parameters

fp = torch.tensor(4*[0.0], dtype=torch.float64)
ms = torch.tensor(8*[0.0], dtype=torch.float64)
dp = torch.tensor([0.0], dtype=torch.float64)

In [7]:
# Define parametric chomaticity function

# Compute parametric fixed point (first order dispersion)

pfp, *_ = parametric_fixed_point((0, 1), fp, [ms, dp], ring)
chop(pfp)

# Define ring around parametric fixed point

def mapping(state, ms, dp):
    return ring(state + evaluate(pfp, [ms, dp]), ms, dp) - evaluate(pfp, [ms, dp])

# Define tunes

def tune(ms, dp):
    matrix = torch.func.jacrev(mapping)(fp, ms, dp)
    tunes, *_ = twiss(matrix)
    return tunes

# Define chromaticity

def chromaticity(ms):
    return torch.func.jacrev(tune, 1)(ms, dp).squeeze()

# Compute natural chromaticity

print(chromaticity(ms))

tensor([-2.0649, -0.8260], dtype=torch.float64)


In [8]:
# Chromaticity can be corrected in a single step

# Compute starting values

psix, psiy = chromaticity(ms)

# Set target values

psix_target = torch.tensor(5.0, dtype=torch.float64)
psiy_target = torch.tensor(5.0, dtype=torch.float64)

# Perform correction

dpsix = psix - psix_target
dpsiy = psiy - psiy_target

solution = - torch.linalg.pinv((torch.func.jacrev(chromaticity)(ms)).squeeze()) @ torch.stack([dpsix, dpsiy])
print(solution)

# Test solution

print(chromaticity(solution))

tensor([ 0.7439, -1.2084,  0.7439, -1.2084,  0.7439, -1.2084,  0.7439, -1.2084], dtype=torch.float64)
tensor([5.0000, 5.0000], dtype=torch.float64)


In [9]:
# Optimization (wrapping objective funtion and normalization)

# Set model parameters
# Parameters are not cloned inside the module on initialization, values will change during optimization!

ms = torch.tensor(8*[0.0], dtype=torch.float64)
ms, *_ = forward([ms], [(-10, 10)])

# Define scalar objective function

def objective(ms):
    psix, psiy = chromaticity(ms)
    return ((psix - psix_target)**2 + (psiy - psiy_target)**2).sqrt()

print(objective(solution))

# Define normalized objective

objective = normalize(objective, [(-10.0, 10.0)])

print(objective(*forward([solution], [(-10, 10)])))


# Set model (forward returns evaluated objective)

model = Wrapper(objective, ms)

# Set optimizer

optimizer = torch.optim.Adam(model.parameters(), lr=1.0E-3)

# Perfom optimization

epochs = 128
for epoch in range(epochs):

    # Evaluate model
    error = model()
    
    # Compute derivatives
    error.backward()

    # Perform optimization step
    optimizer.step()

    # Set gradient to zero
    optimizer.zero_grad()

    # Verbose
    knobs, *_ = [*model.parameters()]
    knobs, *_ = inverse([knobs], [(-10, 10)])
    print(error.detach(), (knobs.detach() - solution).norm())

tensor(5.6871e-15, dtype=torch.float64)
tensor(1.1580e-14, dtype=torch.float64)
tensor(9.1573, dtype=torch.float64) tensor(2.7830, dtype=torch.float64)
tensor(8.9651, dtype=torch.float64) tensor(2.7280, dtype=torch.float64)
tensor(8.7737, dtype=torch.float64) tensor(2.6732, dtype=torch.float64)
tensor(8.5832, dtype=torch.float64) tensor(2.6184, dtype=torch.float64)
tensor(8.3937, dtype=torch.float64) tensor(2.5637, dtype=torch.float64)
tensor(8.2052, dtype=torch.float64) tensor(2.5090, dtype=torch.float64)
tensor(8.0177, dtype=torch.float64) tensor(2.4545, dtype=torch.float64)
tensor(7.8314, dtype=torch.float64) tensor(2.4000, dtype=torch.float64)
tensor(7.6464, dtype=torch.float64) tensor(2.3456, dtype=torch.float64)
tensor(7.4627, dtype=torch.float64) tensor(2.2914, dtype=torch.float64)
tensor(7.2804, dtype=torch.float64) tensor(2.2373, dtype=torch.float64)
tensor(7.0995, dtype=torch.float64) tensor(2.1833, dtype=torch.float64)
tensor(6.9202, dtype=torch.float64) tensor(2.1294, dtype

In [10]:
# Compare

print(solution)
print(*inverse([ms], [(-10, 10)]))

tensor([ 0.7439, -1.2084,  0.7439, -1.2084,  0.7439, -1.2084,  0.7439, -1.2084], dtype=torch.float64)
tensor([ 0.7412, -1.2115,  0.7412, -1.2115,  0.7412, -1.2115,  0.7412, -1.2115], dtype=torch.float64)
